# IDW_interpolation

In [ ]:
"""
- Since weather data is not available for all locations nationwide,
  weather data from all ASOS stations is used to interpolate each feature using IDW (Inverse Distance Weighting).
- During interpolation, if even one feature is missing, the entire row is dropped (treated as missing).
- If at least one station provides valid data, interpolation is still performed (to avoid complete exclusion).
  --> As a result, some wildfire samples are interpolated using data from multiple stations, others from only one or two.
"""

print("Start climate interpolation")
filepath = f"Data/Interpolate_climate/"
o_data = pd.read_csv("Data/gangwon_nofire(2011-2022).csv")
os.makedirs(filepath, exist_ok=True)

# There should be exactly 4 folders for 4 features
if len(glob(filepath + '/*')) != 4:

    features = ['humidity', 'wind_sp', 'rainfall', 'temp']

    os.makedirs(filepath + features[0], exist_ok=True)
    os.makedirs(filepath + features[1], exist_ok=True)
    os.makedirs(filepath + features[2], exist_ok=True)
    os.makedirs(filepath + features[3], exist_ok=True)

    weather_data = pd.read_csv(f"Data/final_climate_data.csv", encoding='cp949')
    weather_data.columns = ['num', 'loc_info', 'lon', 'lat', 'time', 'humidity', 'wind_sp', 'rainfall', 'temp']
    
    # Drop rows where 'time' is missing (indicates failed crawling)
    weather_data.dropna(subset=['time'], inplace=True)

    # Fill missing rainfall values with 0 (assumed no rainfall)
    weather_data['rainfall'] = weather_data['rainfall'].fillna(0)

    for i in tqdm(range(len(o_data))):
        tmp = weather_data[weather_data['num'] == i]
        tmp = tmp.dropna()

        # If all values are missing after drop, skip interpolation
        if len(tmp) == 0:
            print("There is no data")
            continue

        tmp.drop(['num', 'loc_info', 'time'], axis=1, inplace=True)
        tmp = gpd.GeoDataFrame(tmp, geometry=gpd.points_from_xy(tmp.lon, tmp.lat))
        tmp.to_file(f'data{i}.shp')

        for j in range(len(features)):
            idw.idw_interpolation(
                input_point_shapefile=f'data{i}.shp',           # Shapefile of input points to interpolate
                extent_shapefile="Data/gw_boundary/boundary.shp",  # Boundary shapefile (currently Gangwon Province)
                column_name=features[j],                       # Feature name to interpolate
                power=2,                                       # Power parameter for distance weighting
                search_radious=8,                              # Search radius
                output_resolution=400,                         # Resolution of the output raster
            )
            image = rasterio.open(f"data{i}_idw.tif")
            image = pd.DataFrame(image.read(1))
            image.to_csv(f"{filepath}{features[j]}/data{i}_idw.csv", encoding='cp949')
            os.remove(f"data{i}_idw.tif")

        # Remove temporary shapefiles
        os.remove(f"data{i}.shp")
        os.remove(f"data{i}.cpg")
        os.remove(f"data{i}.dbf")
        os.remove(f"data{i}.shx")
        print("-->Complete")
        print("---------------")

else:
    print("-->Already existed")
    print("---------------")


In [9]:
print("Start find fire loc")
filepath = f"Data/fire_loc/"
os.makedirs(filepath, exist_ok=True)

# Only run if the fire location folder is empty
if len(glob(filepath + '/*')) == 0:
    for i in tqdm(range(len(o_data))):
        tmp = pd.DataFrame(o_data.iloc[i])
        tmp = tmp.T
        tmp = gpd.GeoDataFrame(tmp, geometry=gpd.points_from_xy(tmp.lon, tmp.lat))
        tmp.drop(['date', 'time', 'lon', 'lat', 'input'], axis=1, inplace=True)
        tmp.to_file('fire_data.shp')

        shp_path = "fire_data.shp"               # Temporary shapefile with fire point
        boundary_path = 'Data/gw_boundary/boundary.shp'  # Boundary shapefile
        tif_path = f"fire_data.tif"              # Output raster file name

        driver = gdal.GetDriverByName('GTiff')

        # Load shapefile containing fire location
        shp_datasource = gdal.OpenEx(shp_path, gdal.OF_VECTOR)
        # Load Gangwon boundary shapefile
        boundary_datasource = gdal.OpenEx(boundary_path, gdal.OF_VECTOR)

        # Create output raster (400x278 resolution, 1 band, Float32 type)
        tif_datasource = driver.Create(tif_path, 400, 278, 1, gdal.GDT_Float32)

        # Set projection based on shapefile
        tif_datasource.SetProjection(shp_datasource.GetProjection())

        # Get extent (bounding box) of Gangwon Province
        boundary = gpd.read_file(boundary_path)
        xmin, ymin, xmax, ymax = boundary.total_bounds

        # Set geotransform (map the bounding box to 400x278 grid)
        tif_datasource.SetGeoTransform((xmin, (xmax - xmin) / 400, 0, ymax, 0, -(ymax - ymin) / 278))

        band = tif_datasource.GetRasterBand(1)
        
        # Fill entire raster with default no-data value 32767
        band.Fill(32767)
        band.SetNoDataValue(32767)

        # Fill Gangwon interior with -9999
        gdal.RasterizeLayer(tif_datasource, [1], boundary_datasource.GetLayer(), burn_values=[-9999], options=["ALL_TOUCHED=TRUE"])

        # Mark fire location with value 624
        gdal.RasterizeLayer(tif_datasource, [1], shp_datasource.GetLayer(), burn_values=[624])

        # Close raster and shapefile resources
        shp_datasource = None
        tif_datasource = None

        # Read raster data and convert to CSV
        tif_file = rasterio.open(f"fire_data.tif") 
        data = tifffile.imread(f"fire_data.tif")    
        data = pd.DataFrame(data)
        data.to_csv(f"{filepath}fire_data{i}.csv")
        tif_file.close()

    # Remove temporary shapefiles and raster
    os.remove(f"fire_data.shp")
    os.remove(f"fire_data.cpg")
    os.remove(f"fire_data.dbf")
    os.remove(f"fire_data.shx")
    os.remove(f"fire_data.tif")
    print("-->Complete")
    print("---------------")
else:
    print("-->Already existed")
    print("---------------")


Start find fire loc


  0%|                                                                                         | 0/1053 [00:00<?, ?it/s]

0

0

0

...

0

0

0

100%|██████████████████████████████████████████████████████████████████████████████| 1053/1053 [08:49<00:00,  1.99it/s]

-->Complete
---------------


In [10]:
print("Start get climate information")
filepath=f"Data/train_nofire.csv"
o_data=pd.read_csv("Data/gangwon_nofire(2011-2022).csv")

if os.path.isfile(filepath):
    print("Already existed")
    print("---------------")
else:
    climate=[]
    for i in tqdm(range(len(o_data))):
        data=pd.read_csv(f"Data/fire_loc/fire_data{i}.csv")
        index = data[data == 624].stack().index[0]
        temp=pd.read_csv(f"Data/Interpolate_climate/temp/data{i}_idw.csv")
        rain=pd.read_csv(f"Data/Interpolate_climate/rainfall/data{i}_idw.csv")
        hums=pd.read_csv(f"Data/Interpolate_climate/ver2/humidity/data{i}_idw.csv")
        wind=pd.read_csv(f"Data/Interpolate_climate/wind_sp/data{i}_idw.csv")

        r,c=index[0],int(index[1])
        tmp_data = [
                [a, b, c, d]
                for a, b, c, d in zip(
                    [temp.iloc[r][c]],
                    [rain.iloc[r][c]],
                    [hums.iloc[r][c]],
                    [wind.iloc[r][c]]
                )
            ]
        climate.append(tmp_data[0])
    climate=pd.DataFrame(climate,columns=['기온','강수','습도','풍속'])
    o_data.drop('input',axis=1,inplace=True)
    trainfire=pd.concat([o_data,climate],axis=1,ignore_index=True)
    trainfire.columns=['date','time','lon','lat','temp','rainfall','humidity','windspeed']  
    trainfire.to_csv(filepath,index=False)
    print("-->Complete")
    print("---------------")

'\n- 보간한 기상데이터에서 산불발생위치의 기상 데이터만을 가져옴.\n'

Start get climate information


100%|██████████████████████████████████████████████████████████████████████████████| 1053/1053 [02:58<00:00,  5.90it/s]


-->Complete
---------------


# Image_preprocessing

In [1]:
import os 
import re
import time
import shutil
import requests
import natsort
import json
import random
from glob import glob
from datetime import datetime, timedelta

import warnings
warnings.filterwarnings(action='ignore')
warnings.simplefilter(action='ignore', category=FutureWarning) # FutureWarning 제거

from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = 'all'

import numpy as np
import pandas as pd
from tqdm import tqdm
from matplotlib import pyplot as plt
import matplotlib.colors as colors
from PIL import Image, ImageOps

import tifffile
import rasterio
import geopandas as gpd # geo 데이터프레임 만들기 위함
from pyidw import idw # 보간법 실행
from osgeo import gdal # 래스터 자르기 anaconda로 설치
from rasterio.plot import show # 래스터 시각화
from shapely.geometry import Point
from rasterio.features import geometry_mask
from rasterio.transform import from_origin

In [2]:
from tqdm import tqdm
from osgeo import gdal, ogr
import geopandas as gpd
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import os
from pyproj import Proj, Transformer
import rasterio
import natsort
from glob import glob
import cv2

In [3]:
def crop_image_using_coordinates(InputImage, OutputImage, RefImage, latitude, longitude, CropSize):\
    
    Image = gdal.Open(RefImage, gdal.GA_ReadOnly)
    width = Image.RasterXSize
    height = Image.RasterYSize
    Image = None

    """extract vertex coordinates"""
    rds = rasterio.open(RefImage)
    rds.bounds
    left = rds.bounds[0]
    right = rds.bounds[2]
    top = rds.bounds[3]
    bottom = rds.bounds[1]

    resolution_x = (right - left) / width
    resolution_y = (top - bottom) / height
    print(resoultion_x)
    print(resoultion_y)

    InputCrs = 'EPSG:4326'
    OutputCrs ='EPSG:4326'
    transformer = Transformer.from_crs(InputCrs, OutputCrs)
    longitude, latitude = transformer.transform(longitude, latitude)
    
    left_box = latitude - (resolution_x * CropSize)
    top_box = longitude + (resolution_y * CropSize)
    right_box = latitude + (resolution_x * CropSize)
    bottom_box = longitude - (resolution_y * CropSize)
    window = (left_box, top_box, right_box, bottom_box)

    gdal.Translate(OutputImage, InputImage, projWin = window)

In [4]:
def ndvi_filtering(o_data,crop_size,filepath):
    RefImage = 'Data/gw_boundary/boundary_blank_resized.tif'
    for i in tqdm(range(len(o_data))):
        
        filename=str(o_data['date'][i])[:-2]
        
        year = filename[:4]
        month = int(filename[4:])
    
        if 3 <= month <= 5: month = 4 
        elif 6 <= month <= 9:month = 8
        elif 10 <= month <= 11: month = 11
        else: month = 1
        
        filename=year+str(month).zfill(2)
        
        InputImage = f'Data/Raw_data/geo_data/{data_n}_gw.tif'
        OutputImage = filepath+'/Crop_NDVI_'+str(i)+'.tif'
        
        lon=o_data['lon'][i]
        lat=o_data['lat'][i]
        
        crop_image_using_coordinates(InputImage, OutputImage, RefImage, lon, lat, crop_size)

'\n    NDVI는 년도별, 계절별로 차이가 존재\n\n'

In [5]:
def crop_train(data_n,o_data,crop_size):
    filepath=f"Data/firedata/crop/train/{data_n}"
    os.makedirs(filepath, exist_ok=True)
    
    print(f"Start get {data_n} information")
    tmp=natsort.natsorted(glob(filepath+"/*.tif"))
    
    if(len(tmp)==len(o_data)):
        print(f"----{data_n} train crop image already existed")
        return
    else:
        print(f"Create {data_n} train crop image start")

    if(data_n=="NDVI"):
        ndvi_filtering(o_data,1,filepath)
        print(f"Complete")
        return 
    
    InputImage = f'Data/Raw_data/geo_data/{data_n}_gw.tif'
    RefImage = 'data/gw_boundary/boundary_blank_resized.tif'
    
    for i in tqdm(range(len(o_data))):
        OutputImage = filepath+'/Crop_'+data_n+'_'+str(i)+'.tif'
        
        lon=o_data['lon'][i]
        lat=o_data['lat'][i]
        
        crop_image_using_coordinates(InputImage, OutputImage, RefImage, lon, lat, crop_size)
    print(f"Complete")

In [6]:
def crop_val(data_n,o_data,crop_size):
    filepath=f"Data/firedata/crop/val/{data_n}"
    os.makedirs(filepath, exist_ok=True)
    
    print(f"Start get {data_n} information")
    tmp=natsort.natsorted(glob(filepath+"/*.tif"))
    
    if(len(tmp)==len(o_data)):
        print(f"----{data_n} train crop image already existed")
        return
    else:
        print(f"Create {data_n} train crop image start")
        
    
    if(data_n=="NDVI"):
        ndvi_filtering(o_data,1,filepath)
        print(f"Complete")
        return 
    
    InputImage = f'Data/Raw_data/geo_data/{data_n}_gw.tif'
    RefImage = 'data/gw_boundary/boundary_blank_resized.tif'
    
    for i in tqdm(range(len(o_data))):
        OutputImage = filepath+'/Crop_'+data_n+'_'+str(i)+'.tif'
        
        lon=o_data['lon'][i]
        lat=o_data['lat'][i]
        
        crop_image_using_coordinates(InputImage, OutputImage, RefImage, lon, lat, crop_size)
    print(f"Complete")

In [7]:
def crop_test(data_n,crop_size,width_num,height_num):
    
    filepath=f"Data/firedata/crop/test/{data_n}"
    os.makedirs(filepath, exist_ok=True)
    
    print('#'*20)
    print(f"Start get {data_n} information")
    tmp=natsort.natsorted(glob(filepath+"/*.tif"))
    
    if(len(tmp)==111200):
        print(f"----{data_n} test crop image already existed")
        return
    else:
        print(f"Create {data_n} test crop image start")
    
    if(data_n=='NDVI'):InputImage = f'Data/geo_data/NDVI/202204.tif'   
    else:InputImage = f'Data/Raw_data/geo_data/{data_n}_gw.tif'   
    
    RefImage = 'Data/gw_boundary/boundary_blank_resized.tif'

    num=0

    for lat in tqdm(height_num):
        for lon in tqdm(width_num):
            OutputImage = filepath+'/Crop_'+data_n+'_'+str(num)+'.tif'
            num=num+1
            crop_image_using_coordinates(InputImage, OutputImage, RefImage, lon, lat, crop_size)
    print(f"Complete")

In [14]:
def convert_npy(data_n,types,image_size):
    filepath=f"Data/firedata/npy{image_size}/{types}/"
    os.makedirs(filepath, exist_ok=True)
    
    if os.path.isfile(filepath+f'{data_n}_{types}.npy'):
        print(f"----{data_n}_{types}.npy file already existed")
        return
    
    print(f"Create {data_n}_{types}.npy start")
    
    files=natsort.natsorted(glob(f"Data/firedata/crop/{types}/{data_n}/*.tif"))

    tif_list=[]
    for i in range(len(files)):
        tmp = cv2.imread(files[i], cv2.IMREAD_COLOR)
        tmp = cv2.cvtColor(tmp, cv2.COLOR_BGR2RGB)
        tmp = cv2.resize(tmp, (image_size, image_size))
        #tmp = tmp / 255.0
        tif_list.append(tmp)

    data=np.array(tif_list)

    np.save(filepath+f'{data_n}_{types}.npy',data)
    print("Complete")

In [15]:
def make_train_image_data(cropsize,imagesize):
    print("Train land_dataset create start")
    
    train_data=pd.read_csv("Modeling_data/train/climate_train.csv")
    
    data_n=['Height','Slope','Landuse','population_density','NDVI']
    
    for i in range(len(data_n)):
        print("#"*20)
        crop_train(data_n[i],train_data,cropsize)
        convert_npy(data_n[i],'train',imagesize)
        
    print("Train land_dataset create complete")
    print("#"*30)
    
def make_val_image_data(cropsize,imagesize):
    print("Train land_dataset create start")
    
    val_data=pd.read_csv("Modeling_data/train/climate_val.csv")
    
    #data_n=['Height','Slope','Landuse','population_density','NDVI']
    data_n=['Landuse']
    
    for i in range(len(data_n)):
        print("#"*20)
        crop_val(data_n[i],val_data,cropsize)
        convert_npy(data_n[i],'val',imagesize)
        
    print("Train land_dataset create complete")
    print("#"*30)
        
def make_test_image_data(cropsize,imagesize):
    
    print("Test land_dataset create start")
    #! 강원도 경계
    N = 38.61370931
    E = 129.359995
    S = 37.03353708
    W = 127.0950376

    width = (E-W)/399
    height= (N-S)/277

    width_num,height_num=[], []
    for i in range(400):
        width_num.append(round(W+width*i,7))
    for i in range(278):
        height_num.append(round(N-height*i,7))
    
    data_n=['Height','Slope','Landuse','population_density','NDVI']
    
    for i in range(len(data_n)):   
        print("#"*20)
        crop_test(data_n[i],cropsize,width_num,height_num)
        convert_npy(data_n[i],'test',imagesize)

    print("Test land_dataset create complete")
    

In [ ]:
make_train_image_data(1,24)
make_val_image_data(1,24)
make_test_image_data(1,24)

# Train_val_split

In [11]:
import pandas as pd
from sklearn.model_selection import train_test_split

# Load the datasets
nofire = pd.read_csv("Data/train_nofire.csv")
nofire['target'] = 0

fire = pd.read_csv("Data//train_fire.csv")
fire['target'] = 1

# Combine the datasets
combined_data = pd.concat([nofire, fire], ignore_index=True)

# Split the data into train (80%) and test (20%)
train_data, test_data = train_test_split(combined_data, test_size=0.2, random_state=42, stratify=combined_data['target'])

# Save the split datasets
train_data.to_csv("Modeling_data/train/climate_train.csv", index=False)
test_data.to_csv("Modeling_data/val/climate_val.csv", index=False)

((1684, 9), (422, 9))